In [6]:
from skimage.color import rgb2gray, rgb2hsv
from matplotlib.pyplot import bar
from skimage.exposure import histogram
from skimage.util import random_noise
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.filters import median
from skimage.filters import gaussian
from skimage.color import rgb2gray,rgb2hsv
# import the opencv library
import cv2
#new
from scipy import ndimage
from datetime import datetime
import numpy as np
import math

# Show the figures / plots inside the notebook
%matplotlib inline


In [7]:

def show_images(images, titles=None):
    n_ims = len(images)
    if titles is None:
        titles = ['(%d)' % i for i in range(1, n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image, title in zip(images, titles):
        a = fig.add_subplot(1, n_ims, n)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()


In [9]:

import cv2 
import numpy as np
import math
import skimage.io as io

wait = 0
i = 0
arr=[]
# define a video capture object
vid = cv2.VideoCapture(0)

while (True):

#    # Capture the video frame
#     # by frame
#     ret, frame = vid.read()
#     dim = (1000, 800)
#     frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
#     # Display the resulting frame
#     frame = cv2.flip(frame, 1)
   
#     print(frame.shape)
#     border=frame[500:990, 400:790]
#      #blur the image
#     frame = cv2.GaussianBlur(frame,(5,5),100) 
#     frame = cv2.medianBlur(frame,7)
#     cv2.rectangle(frame,(500,400),(990,790),(0, 0, 0),2)
#     hsv = cv2.cvtColor(border, cv2.COLOR_BGR2HSV)
#     # define range of skin color in HSV
#     # lower_skin = np.array([0,20,70], dtype=np.uint8)
#     # upper_skin = np.array([20,255,255], dtype=np.uint8)
#     lower_skin = np.array([0,15,0], dtype=np.uint8)
#     upper_skin = np.array([17,170,255], dtype=np.uint8)
        
#      #extract skin colur imagw  
#     mask = cv2.inRange(hsv, lower_skin, upper_skin)
#     kernel = np.ones((3,3),np.uint8)
   
        
   
        
#     # #extrapolate the hand to fill dark spots within
#     mask = cv2.dilate(mask,kernel,iterations = 5)
#     # mask = cv2.erode(mask,kernel,iterations = 1)   
# cv2.imshow('frame',frame)
    ret, frame = vid.read()
    dim = (1000, 800)
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    # Display the resulting frame
    frame = cv2.flip(frame, 1)
    # Get hand data from the rectangle sub window   
    cv2.rectangle(frame,(500,400),(990,790),(0, 0, 0),2)
    crop_image = frame[500:990, 400:790]
    
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(crop_image, (3,3), 0)
    
    # Change color-space from BGR -> HSV
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    
    # Create a binary image with where white will be skin colors and rest is black
    mask2 = cv2.inRange(hsv, np.array([0,0,0]), np.array([20,255,255]))
    
    # Kernel for morphological transformation    
    kernel = np.ones((5,5))
    
    # Apply morphological transformations to filter out the background noise
    dilation = cv2.dilate(mask2, kernel, iterations = 1)
    erosion = cv2.erode(dilation, kernel, iterations = 1)    
       
    # Apply Gaussian Blur and Threshold
    mask = cv2.GaussianBlur(erosion, (3,3), 0)
    
    
    #detection
    #find contours

# binaryHand = 255 - binaryHand;
# contours,hierarchy= cv2.findContours(binaryHand,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# # print(contours)

    ret, thresh = cv2.threshold(mask, 127, 255,0)
    contours, hierarchy = cv2.findContours(thresh,2,1)
    if len(contours)!=0:
        #find contour of max area(hand)
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        # print(max(contours, key = lambda x: cv2.contourArea(x)))


        #approx the contour a little
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx= cv2.approxPolyDP(cnt,epsilon,True)
        #approx=cnt
        # cv2.imshow('binaryHand', binaryHand);
        # cv2.waitKey(0)
        # io.imshow(binaryHand)
        # io.show()

        #make convex hull around hand
        hull = cv2.convexHull(cnt)
        print(cnt)
        #define area of hull and area of hand
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)

    #find the percentage of area not covered by hand in convex hull
        if areacnt!=0:
            arearatio=((areahull-areacnt)/areacnt)*100
        else :
            arearatio=0    
        #find the defects in convex hull with respect to hand
        hull = cv2.convexHull(approx, returnPoints=False)
        if len(hull)!=0:
            defects = cv2.convexityDefects(approx, hull)
            # print(defects.shape)
            l=0   
        #     if True:
            if len(defects)!=0:

                #code for finding no. of defects due to fingers
                for i in range(defects.shape[0]):
                    s,e,f,d = defects[i,0]
                    start = tuple(approx[s][0])
                    end = tuple(approx[e][0])
                    far = tuple(approx[f][0])
                    pt= (100,180)


                    # find length of all sides of triangle
                    a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    s = (a+b+c)/2
                    area_triangle  = math.sqrt(s*(s-a)*(s-b)*(s-c))

                    #distance between point and convex hull
                    d=(2*area_triangle)/a

                    # apply cosine rule here
                    angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
                    print(angle)        

                    # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
                    if angle <= 90 and d>30:
                        l += 1
                        #cv2.circle(img, far, 7, [255,0,0], -1)

                    #draw lines around hand
                    cv2.line(crop_image,start, end, [0,255,0], 2)
                    #if(i == 0):
                    cv2.circle(crop_image, start, 4, [255,0,0], -1)
                    cv2.circle(crop_image, end, 4, [0,0,255], -1)#[0,0,255]:red
                    cv2.circle(crop_image, far, 3, [0,255,0], -1)
            #cv2.imshow('final_img',img)
            #cv2.waitKey(0)
            #time.sleep(6)
            #if(i == 0):
                #cv2.circle(img, start, 4, [0,0,255], -1)
                #cv2.circle(img, end, 4, [0,0,0], -1)    


                    l+=1
            # cv2.imshow('final_img',img)
            # cv2.waitKey(0)
            font = cv2.FONT_HERSHEY_SIMPLEX
            # frame=img
            if l==1:
            #     if areacnt<2000:
            #         cv2.putText(frame,'Put hand in the box',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            #     else:
            #     if arearatio<12:
            #         cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            #     elif arearatio<17.5:
            #         cv2.putText(frame,'Best of luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            #     else:
                cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==2:
                cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==3:

            #     if arearatio<27:
                cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            #     else:
            #         cv2.putText(frame,'ok',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==4:
                cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==5:
                cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==6:
                cv2.putText(frame,'reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            else :
                cv2.putText(frame,'reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)



    cv2.imshow('frame',frame)        
    cv2.imshow('mask',mask)  

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
   
    # cv2.imshow('frame',frame)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    # wait variable is to calculate waiting time
    # key = cv2.waitKey(100)
    # wait = wait+100
    
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break
    # if key == ord('q'):
    #     break
    # if wait == 5000:
    #     arr[i]=mask
    #     i = i+1
        # cv2.imshow('mask',mask)

# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()


[[[0 0]]

 [[0 1]]

 [[0 2]]

 ...

 [[3 0]]

 [[2 0]]

 [[1 0]]]
82.81950781487889
104.0326668963346
76.38955459403789
125.03096544651605
113.37417741332509
67.74289088246756
50.05691419589443
[[[0 0]]

 [[0 1]]

 [[0 2]]

 ...

 [[3 0]]

 [[2 0]]

 [[1 0]]]
55.732659274062506
155.61853973174624
63.30946047705109
107.24809851142918
95.84285223422116
78.70605821594232
112.0868715353467
[[[0 0]]

 [[0 1]]

 [[0 2]]

 ...

 [[3 0]]

 [[2 0]]

 [[1 0]]]
56.76595134749447
127.16221334818765
91.5841596114629
71.19560902670051
119.92626589387828
67.03229680442273
138.67908374334095
86.78253295398568
126.21495382852633
67.24332343441533
66.48886388211245
[[[0 0]]

 [[0 1]]

 [[0 2]]

 ...

 [[3 0]]

 [[2 0]]

 [[1 0]]]
63.47366905930242
132.16276863630748
73.56425726063536
117.28864684055144
30.160275133683434
86.53815611548544
78.9300745298159
83.36471892120261
64.61454339849398
113.37417741332517
[[[0 0]]

 [[0 1]]

 [[0 2]]

 ...

 [[3 0]]

 [[2 0]]

 [[1 0]]]
53.93015083378916
86.85217992

[[[153   0]]

 [[153   1]]

 [[153   2]]

 ...

 [[156   0]]

 [[155   0]]

 [[154   0]]]
95.98499657796673
157.8573784296273
94.12223826944037
100.78693753875231
141.39132063012906
80.2522723277354
104.71175742891187
114.1252320415456
102.7432505751648
152.64286754157231
87.83946267224519
97.49528898533288
47.23874887253988
152.64286754157226
41.46130422588245
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   3]]

 [[100   2]]

 [[100   1]]]
58.907812656097526
129.77797140878633
65.94544663634456
131.17124134088968
105.59638706189018
132.89596444379447
139.14248816759783
83.22792902039706
77.67949840954053
103.49917442554036
26.73073291410761
[[[90  0]]

 [[90  1]]

 [[91  2]]

 ...

 [[93  0]]

 [[92  0]]

 [[91  0]]]
25.783496725544335
105.05205189341048
109.25951536574195
58.268074687628996
131.51631764234423
122.5153996383618
79.75870887754449
32.72176108743371
[[[216 107]]

 [[216 108]]

 [[215 109]]

 ...

 [[219 108]]

 [[218 107]]

 [[217 107]]]
110.20041215138605
101.9

[[[96  0]]

 [[96  1]]

 [[96  2]]

 ...

 [[99  0]]

 [[98  0]]

 [[97  0]]]
117.23132750920455
148.4055214348133
104.71175742891187
170.38969263908666
107.8751722279177
94.65744856563045
23.264612623777047
[[[ 97   0]]

 [[ 97   1]]

 [[ 97   2]]

 ...

 [[100   0]]

 [[ 99   0]]

 [[ 98   0]]]
83.9186599380534
126.21495382852633
114.67240539627102
57.608471728855704
164.44142261210206
101.99952054212379
90.2612396130378
26.015583670131335
[[[ 99   0]]

 [[ 99   1]]

 [[ 99   2]]

 ...

 [[100   2]]

 [[100   1]]

 [[100   0]]]
102.46916607004557
115.96330434035505
75.66860049609505
125.16966296187765
47.60786216020005
166.9167186011057
127.21485125179827
[[[ 99   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   2]]

 [[100   1]]

 [[100   0]]]
90.12502476700064
63.747442893502466
133.38295804187004
40.973309978432596
163.62481634875746
107.8751722279177
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[101   3]]

 [[100   2]]

 [[100   1]]]
114.96340691708312
56.87632048265286
100.7

[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   3]]

 [[100   2]]

 [[100   1]]]
90.32772462143144
96.6236253164745
89.5353906273091
81.51188433752034
138.7319676001324
101.99952054212388
[[[ 99   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   2]]

 [[100   1]]

 [[100   0]]]
90.12502476700064
119.18607865968836
83.77404853080137
86.49557114021843
84.4064211187425
149.62357380080363
69.75893854306477
162.4577439693799
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[103   0]]

 [[102   0]]

 [[101   0]]]
100.78693753875231
115.48026505638572
106.94723364705739
110.7836831863531
128.4006871812366
100.45935493844961
162.45774396937972
95.70274219473443
[[[96  0]]

 [[96  1]]

 [[96  2]]

 ...

 [[99  0]]

 [[98  0]]

 [[97  0]]]
74.83480576395117
65.47307557538173
139.40631700866558
163.9235903590155
142.74635364496925
92.69880542631498
85.66534831501463
34.87524561262974
[[[95  0]]

 [[95  1]]

 [[95  2]]

 ...

 [[98  0]]

 [[97  0]]

 [[96  0]]]
40.615849596955336
138.2004

[[[233   0]]

 [[233   1]]

 [[233   2]]

 ...

 [[236   0]]

 [[235   0]]

 [[234   0]]]
57.34568196837949
120.63772873534766
103.08510355309873
33.87197678811076
86.74697041812519
85.15939282493176
117.37042583752424
101.20203177222481
103.49917442554036
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   3]]

 [[100   2]]

 [[100   1]]]
93.31518448852941
84.20721410126949
114.65867458615175
73.68392849027349
84.90933397799554
163.89042741671898
83.22792902039706
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   3]]

 [[100   2]]

 [[100   1]]]
93.79720665403792
101.81077951008018
143.69411040295716
129.15384958643466
111.57949510867203
103.49917442554042
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   3]]

 [[100   2]]

 [[100   1]]]
100.25495298564867
123.72825454538987
95.8719487094887
149.826480418303
107.50419279263875
145.55463285240685
134.3185205465978
101.99952054212373
[[[100   0]]

 [[100   1]]

 [[100   2]]

 ...

 [[100   3]]

 [[100   2]]

 [[100   

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'cv::convexityDefects'
